In [3]:

import schnetpack as spk
import torch
import schnetpack.nn 
import schnetpack.data
from schnetpack.datasets import QM9

from ase.io import read
import numpy as np
from numpy import genfromtxt, savetxt

In [4]:
### #Loads the best_model after schnet training.
#user evaluates the model on the QM9 database
model_file = "../../../../data/trained_models/qm9_i3_30f_10000_5000/best_model"
split_file = '../../../../data/trained_models/qm9_i3_30f_10000_5000/split.npz'
idx=1
# Load best model where it was stored after training

model = torch.load(model_file, map_location=torch.device('cpu'))

# Download QM9 dataset to use in evaluating model
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)

# Load split file 
train, val, test = spk.data.train_test_split(qm9data, split_file=split_file)


device = 'cpu'

# load data for molecule
at, props = qm9data.get_properties(idx)
# load spk calculator
calculator = spk.interfaces.SpkCalculator(model=model, device=device, energy=QM9.mu)
# set calculator on molecule
at.set_calculator(calculator)

converter = spk.data.AtomsConverter(device=device)
# convert qm9 data to machine-readable form
inputs = converter(at)


pred = model(inputs)

print('Prediction:', pred[QM9.U0].detach().cpu().numpy()[0,0])
print('Keys:', list(inputs.keys()))
print('Truth:', props[QM9.U0].cpu().numpy()[0])

Prediction: -1538.8138
Keys: ['_atomic_numbers', '_positions', '_cell', '_neighbors', '_cell_offset', '_atom_mask', '_neighbor_mask', 'representation']
Truth: -1538.1477
